<h1>Implementacja algorytmu mrówkowego.</h1>

In [ ]:
from os import listdir, path
import re

import time
import numpy as np
import random
import math





BASE_DIR_PATH = path.dirname(path.abspath("__file__"))  # gdyby nie działało, usuń cudzysłów
INSTANCES_PATH = path.join(BASE_DIR_PATH, "instances")
NEGATIVE_RANDOM_PATH = path.join(INSTANCES_PATH, "negative_random")
NEGATIVE_REPETITIONS_PATH = path.join(INSTANCES_PATH, "negative_repetitions")
POSITIVE_END_ERRORS_PATH = path.join(INSTANCES_PATH, "positive_end_errors")
POSITIVE_RANDOM_PATH = path.join(INSTANCES_PATH, "positive_random")





negative_random_filenames = [path.join(NEGATIVE_RANDOM_PATH, f) for f in listdir(NEGATIVE_RANDOM_PATH) if path.isfile(path.join(NEGATIVE_RANDOM_PATH, f))]
negative_repetitions_filenames = [path.join(NEGATIVE_REPETITIONS_PATH, f) for f in listdir(NEGATIVE_REPETITIONS_PATH) if path.isfile(path.join(NEGATIVE_REPETITIONS_PATH, f))]
positive_end_errors_filenames = [path.join(POSITIVE_END_ERRORS_PATH, f) for f in listdir(POSITIVE_END_ERRORS_PATH) if path.isfile(path.join(POSITIVE_END_ERRORS_PATH, f))]
positive_random_filenames = [path.join(POSITIVE_RANDOM_PATH, f) for f in listdir(POSITIVE_RANDOM_PATH) if path.isfile(path.join(POSITIVE_RANDOM_PATH, f))]

selected_filenames = negative_random_filenames + negative_repetitions_filenames + positive_end_errors_filenames + positive_random_filenames





class ProblemPropertiesClass:
    def __init__(self):
        self.words = []
        self.word_len = 0
        self.org_seq_len = 0
        self.words_num = 0
        
        self.common_nucleotides_matrix = np.zeros((self.words_num + 1, self.words_num + 1), dtype=int)
        self.wages = np.zeros(self.word_len)
        self.next_word_power_matrix = np.zeros((self.words_num + 1, self.words_num + 1))
        
        self.sequences = []
        self.best_sequence = []
        self.best_sequence_len = 0
        self.sequences_lengths_of_all_iterations = []
        
        self.avaible_words = np.ones(self.words_num + 1, dtype=bool)
        self.avaible_words_number = self.words_num
        self.current_word = 0
        self.current_length = 0
        self.sequence = []
        
        self.iteration_with_the_best_sequence = 0
        self.iterations_number_limit = 30
        
        #parametry algorytmu
        self.iterations_number = 100
        self.attempts_number = 100
        self.part_of_the_best_sequences = 0.1
        self.best_price_for_sequence = 1.2
        self.k_set_starting_wages = 2
        self.compensation_coef = 0.98
        self.base_of_logarithm = 2
    
    
    def set_instance_info(self, words=[], word_len=0, org_seq_len=0, words_num=0):
        self.words = words[:]
        self.word_len = word_len
        self.org_seq_len = org_seq_len
        self.words_num = words_num
        
        self.common_nucleotides_matrix = np.zeros((self.words_num + 1, self.words_num + 1), dtype=int)
        self.wages = np.zeros(self.word_len)
        self.next_word_power_matrix = np.zeros((self.words_num + 1, self.words_num + 1))
        
        self.avaible_words = np.ones(self.words_num + 1, dtype=bool)
        self.avaible_words_number = self.words_num
        self.current_word = 0
        self.current_length = 0
        self.sequence = []
    
    def prepare_general_vars(self):
        self.common_nucleotides_matrix = np.zeros((self.words_num + 1, self.words_num + 1), dtype=int)
        self.wages = np.zeros(self.word_len)
        self.next_word_power_matrix = np.zeros((self.words_num + 1, self.words_num + 1))
        self.best_sequence = []
        self.sequences_lengths_of_all_iterations = []
        self.iteration_with_the_best_sequence = 0
        
    def prepare_iterations_vars(self):
        self.sequences = []
    
    def prepare_attempts_vars(self):
        self.avaible_words = np.ones(self.words_num + 1, dtype=bool)    #definicja wektora określającego, czy i-ty wyraz jest dostępny
        self.avaible_words_number = self.words_num                      #definicja zmiennej zawierającej liczbę niewykorzystanych jeszcze wyrazów
        self.current_word = 0                                           #definicja zmiennej wskazującej ostatnio wybranego słowa -> słowo zerowe, czyli zaczynane jest tworzenie sekwencji
        self.current_length = 0                                         #definicja zmiennej zawierającej długość utworzonej dotychczas sekwencji
        self.sequence = []
        
        
        
        
        
#funkcja zwraca maksymalną liczbę nukleotydów, na których zazębia się sufiks nucl1 z prefiksem nucl2
#poszukiwanie poprzez coraz to mniejszą długość ciągu, porównywanie ciągów znak po znaku
def common_nucleotides_max_number_full_iterating(nucl1, nucl2):
    word_len = len(nucl1)
    result = word_len - 1       #zmienna zawierająca końcowy wynik
    
    while(result > 0):          #badanie coraz to mniejszej możliwej długości wspólnego ciągu
        i1 = word_len - result
        i2 = 0
        
        while(i1 < word_len):              #porównanie nachodzących końcówek, znak po znaku
            if(nucl1[i1] != nucl2[i2]):    #znaki się nie zgadzają
                break
            i1 += 1
            i2 += 1
        
        if(i1 == word_len):    #porównywanie ciągów zostało wykonane pozytywnie po wszystkich znakach
            break
        
        result -= 1
    
    return result


#funkcja zwraca maksymalną liczbę nukleotydów, na których zazębia się sufiks nucl1 z prefiksem nucl2
#poszukiwanie poprzez coraz to mniejszą długość ciągu, bezpośrednie porównywanie ciągów przy pomocy mechanizmu "slicing"
def common_nucleotides_max_number_iterating_with_slicing(nucl1, nucl2):
    word_len = len(nucl1)
    result = word_len - 1       #zmienna zawierająca końcowy wynik
    
    while(result > 0):          #badanie coraz to mniejszej możliwej długości wspólnego ciągu
        i1 = word_len - result
        i2 = 0
        
        if(nucl1[-result:] == nucl2[:result]): #porównywanie ciągów przy pomocy mechanizmu "slicing"
            break
        
        result -= 1
    
    return result





def generate_common_nucleotides_matrix_full_iterating(problem_properties):
    for i in range(1, problem_properties.words_num + 1):
            for j in range(1, problem_properties.words_num + 1):
                if(i != j):
                    problem_properties.common_nucleotides_matrix[i, j] = common_nucleotides_max_number_full_iterating(problem_properties.words[i], problem_properties.words[j])
                else:
                    problem_properties.common_nucleotides_matrix[i, i] = 0

def generate_common_nucleotides_matrix_with_numpy_tricks_common_nucleotides_max_number(problem_properties, i, j):
    if(i != j):
        return common_nucleotides_max_number_full_iterating(problem_properties.words[i], problem_properties.words[j])
    else:
        return 0

def generate_common_nucleotides_matrix_with_numpy_tricks(problem_properties):
    index_of_column = np.zeros((problem_properties.words_num + 1, problem_properties.words_num + 1), dtype=int)
    index_of_row = np.zeros((problem_properties.words_num + 1, problem_properties.words_num + 1), dtype=int)

    index_of_column[:] = np.arange(problem_properties.words_num + 1)
    index_of_row = index_of_column.transpose()

    vectorized_generate_common_nucleotides_matrix_with_numpy_tricks_common_nucleotides_max_number = np.vectorize(generate_common_nucleotides_matrix_with_numpy_tricks_common_nucleotides_max_number)
    problem_properties.common_nucleotides_matrix[1:, 1:] = vectorized_generate_common_nucleotides_matrix_with_numpy_tricks_common_nucleotides_max_number(problem_properties, index_of_row[1:, 1:], index_of_column[1:, 1:])
    
    
    
    
    
def set_starting_wages(n, k):
    result = np.zeros(n)
    n -= 1
    result[n] = 1.0
    while(n > 0):
        n -= 1
        result[n] = result[n + 1] / k
    
    return result





#funkcja oblicza początkowe wagi prawdopodobieństwa dla wyboru j-tego wyrazu po dokonaniu już wyboru i-tego
#waga przyznawana jest na podstawie długości wspólnego sufiksu-prefiksu słowa i-tego i j-tego
def generate_next_word_power_matrix_1_full_iterating(problem_properties):
    #przydzielenia wag dla konkretnych połączeń wyrazów
    for i in range(1, problem_properties.words_num + 1):
        for j in range(1, problem_properties.words_num + 1):
            problem_properties.next_word_power_matrix[i, j] = problem_properties.wages[ problem_properties.common_nucleotides_matrix[i, j] ]
            
            
#funkcja oblicza początkowe wagi prawdopodobieństwa dla wyboru j-tego wyrazu po dokonaniu już wyboru i-tego
#waga przyznawana jest na podstawie długości wspólnego sufiksu-prefiksu słowa i-tego i j-tego
#szybsza wersja wykorzystująca możliwości biblioteki numpy
def generate_next_word_power_matrix_1_with_numpy_tricks(problem_properties):
    #przydzielenia wag dla konkretnych połączeń wyrazów
    problem_properties.next_word_power_matrix[1:, 1:] = problem_properties.wages[ problem_properties.common_nucleotides_matrix[1:, 1:] ]

    
#funkcja oblicza początkowe wagi prawdopodobieństwa dla wyboru j-tego wyrazu po dokonaniu już wyboru i-tego
#określona już waga prawdopodobieństwa dla połączenia wyrazów o pewnej długości zazębiających się końcówek jest równomiernie rozdzielana na wsystkie połączenia o tejże długości wspólnego ciągu
def generate_next_word_power_matrix_2_full_iterating(problem_properties):
    for i in range(1, problem_properties.words_num + 1):
        #zliczenie wystąpień zachodzenia wyrazów na danej liczbie znaków
        occurrence_counter = np.zeros(problem_properties.word_len)
        for j in range(1, problem_properties.words_num + 1):
            occurrence_counter[ problem_properties.common_nucleotides_matrix[i, j] ] += 1
        
        #obliczenie wartości wag prawdopodobieństwa obowiązujących dla danego i-tego wiersza -> dla poszukiwania kolejnego wyrazu po i-tym wyrazie
        current_wages = np.copy(problem_properties.wages)
        for j in range(problem_properties.word_len):
            if(occurrence_counter[j] != 0):
                current_wages[j] /= occurrence_counter[j]
        
        #przydzielenie wag dla konkretnych połączeń wyrazów
        for j in range(1, problem_properties.words_num + 1):
            problem_properties.next_word_power_matrix[i, j] = current_wages[ problem_properties.common_nucleotides_matrix[i, j] ]

            
#funkcja oblicza początkowe wagi prawdopodobieństwa dla wyboru j-tego wyrazu po dokonaniu już wyboru i-tego
#określona już waga prawdopodobieństwa dla połączenia wyrazów o pewnej długości zazębiających się końcówek jest równomiernie rozdzielana na wsystkie połączenia o tejże długości wspólnego ciągu
#szybsza wersja wykorzystująca możliwości biblioteki numpy
def generate_next_word_power_matrix_2_with_numpy_tricks_1(problem_properties):
    for i in range(1, problem_properties.words_num + 1):
        #zliczenie wystąpień zachodzenia wyrazów na danej liczbie znaków
        occurrence_counter = np.zeros(problem_properties.word_len)
        unique, counts = np.unique(problem_properties.common_nucleotides_matrix[i, 1:], return_counts=True)
        occurrence_counter[unique] = counts
        
        #obliczenie wartości wag prawdopodobieństwa obowiązujących dla danego i-tego wiersza -> dla poszukiwania kolejnego wyrazu po i-tym wyrazie
        current_wages = np.copy(problem_properties.wages)
        occurrence_counter[occurrence_counter == 0] = 1
        current_wages /= occurrence_counter
        
        #przydzielenia wag dla konkretnych połączeń wyrazów
        problem_properties.next_word_power_matrix[i, 1:] = current_wages[ problem_properties.common_nucleotides_matrix[i, 1:] ]

            
#funkcja oblicza początkowe wagi prawdopodobieństwa dla wyboru j-tego wyrazu po dokonaniu już wyboru i-tego
#określona już waga prawdopodobieństwa dla połączenia wyrazów o pewnej długości zazębiających się końcówek jest równomiernie rozdzielana na wsystkie połączenia o tejże długości wspólnego ciągu
#szybsza wersja wykorzystująca możliwości biblioteki numpy
def generate_next_word_power_matrix_2_with_numpy_tricks_2(problem_properties):
    occurrence_counter = np.zeros((problem_properties.words_num + 1, problem_properties.word_len))
    
    for i in range(1, problem_properties.words_num + 1):
        #zliczenie wystąpień zachodzenia wyrazów na danej liczbie znaków
        unique, counts = np.unique(problem_properties.common_nucleotides_matrix[i, 1:], return_counts=True)
        occurrence_counter[i, unique] = counts
        
    occurrence_counter[occurrence_counter == 0] = 1
        
    #obliczenie wartości wag prawdopodobieństwa obowiązujących dla danego i-tego wiersza -> dla poszukiwania kolejnego wyrazu po i-tym wyrazie
    current_wages = np.zeros((problem_properties.words_num+1, problem_properties.word_len))
    current_wages[:] = problem_properties.wages
    current_wages /= occurrence_counter
    
    #przydzielenia wag dla konkretnych połączeń wyrazów
    for i in range(1, problem_properties.words_num + 1):
        problem_properties.next_word_power_matrix[i, 1:] = current_wages[i, problem_properties.common_nucleotides_matrix[i, 1:] ]
        
        
        
        
        
#funkcja ustawia równe szanse wyboru słowa rozpoczynającego sekwencje
def generate_chance_for_first_word(problem_properties):
    problem_properties.next_word_power_matrix[0, 1:] = 1;

#funkcja ustawia większe szanse wyboru słowa dla wyrazów, których prefiks w maksymalnym przypadku pokrywa się na jaknajmniejszej ilości zanków z jakim kolwiek sufiksem
def generate_chance_for_first_word_1_full_iterating(problem_properties):
    max_common_nucleotides = np.zeros(problem_properties.words_num + 1, dtype=int)

    #wyznaczenie maksymalnej długości zazębiającego się prefiksu j-tego wyrazu
    for j in range(1, problem_properties.words_num + 1):
        max_common_nucleotides[j] = problem_properties.common_nucleotides_matrix[1, j]
        for i in range(2, problem_properties.words_num + 1):
            if(max_common_nucleotides[j] < problem_properties.common_nucleotides_matrix[i, j]):
                max_common_nucleotides[j] = problem_properties.common_nucleotides_matrix[i, j]
    
    #przyznanie wagi prawdopodobieństwa przy użyciu ustalonych wag
    for j in range(1, problem_properties.words_num + 1):
        problem_properties.next_word_power_matrix[0, j] = problem_properties.wages[ problem_properties.word_len - 1 - max_common_nucleotides[j] ]

        
#funkcja ustawia większe szanse wyboru słowa dla wyrazów, których prefiks w maksymalnym przypadku pokrywa się na jaknajmniejszej ilości zanków z jakim kolwiek sufiksem
#szybsza wersja wykorzystująca możliwości biblioteki numpy
def generate_chance_for_first_word_1_with_numpy_tricks(problem_properties):
    #wyznaczenie maksymalnej długości zazębiającego się prefiksu j-tego wyrazu
    max_common_nucleotides = problem_properties.common_nucleotides_matrix.max(0)
    
    #przyznanie wagi prawdopodobieństwa przy użyciu ustalonych wag
    problem_properties.next_word_power_matrix[0] = problem_properties.wages[ problem_properties.word_len - 1 - max_common_nucleotides]
    
    

#funkcja ustawia większe szanse wyboru słowa dla wyrazów, których prefiks w maksymalnym przypadku pokrywa się na jaknajmniejszej ilości zanków z jakim kolwiek sufiksem
#określona już waga jest równomiernie rozdzielona na wyrazy o poszczególnej długości maksymalnego zazębienia
def generate_chance_for_first_word_2_full_iterating(problem_properties):
    max_common_nucleotides = np.zeros(problem_properties.words_num + 1, dtype=int)

    #wyznaczenie maksymalnej długości zazębiającego się prefiksu j-tego wyrazu
    for j in range(1, problem_properties.words_num + 1):
        max_common_nucleotides[j] = problem_properties.common_nucleotides_matrix[1, j]
        for i in range(2, problem_properties.words_num + 1):
            if(max_common_nucleotides[j] < problem_properties.common_nucleotides_matrix[i, j]):
                max_common_nucleotides[j] = problem_properties.common_nucleotides_matrix[i, j]
    
    #przyznanie wagi prawdopodobieństwa przy użyciu ustalonych wag, waga jest równomiernie rozdizelona
    occurrence_counter = np.zeros(problem_properties.word_len)
    for j in range(1, problem_properties.words_num + 1):
        occurrence_counter[ max_common_nucleotides[j] ] += 1

    #obliczenie wartości wag prawdopodobieństwa obowiązujących dla danego i-tego wiersza -> dla poszukiwania kolejnego wyrazu po i-tym wyrazie
    current_wages = np.flip(np.copy(problem_properties.wages))
    for j in range(problem_properties.word_len):
        if(occurrence_counter[j] != 0):
            current_wages[j] /= occurrence_counter[j]

    #przydzielenia wag dla konkretnych połączeń wyrazów
    for j in range(1, problem_properties.words_num + 1):
        problem_properties.next_word_power_matrix[0, j] = current_wages[ max_common_nucleotides[j] ]
        
        
#funkcja ustawia większe szanse wyboru słowa dla wyrazów, których prefiks w maksymalnym przypadku pokrywa się na jaknajmniejszej ilości zanków z jakim kolwiek sufiksem
#określona już waga jest równomiernie rozdzielona na wyrazy o poszczególnej długości maksymalnego zazębienia
#szybsza wersja wykorzystująca możliwości biblioteki numpy
def generate_chance_for_first_word_2_with_numpy_tricks(problem_properties):
    #wyznaczenie maksymalnej długości zazębiającego się prefiksu j-tego wyrazu
    max_common_nucleotides = problem_properties.common_nucleotides_matrix.max(0)
    
    #przyznanie wagi prawdopodobieństwa przy użyciu ustalonych wag, waga jest równomiernie rozdizelona
    occurrence_counter = np.zeros(problem_properties.word_len)
    unique, counts = np.unique(max_common_nucleotides[1:], return_counts=True)
    occurrence_counter[unique] = counts

    #obliczenie wartości wag prawdopodobieństwa obowiązujących dla danego i-tego wiersza -> dla poszukiwania kolejnego wyrazu po i-tym wyrazie
    current_wages = np.flip(np.copy(problem_properties.wages))
    occurrence_counter[occurrence_counter == 0] = 1
    current_wages /= occurrence_counter

    #przydzielenia wag dla konkretnych połączeń wyrazów
    problem_properties.next_word_power_matrix[0, 1:] = current_wages[ max_common_nucleotides[1:]]
    
    
    
    
    
#funkcja wybiera kolejny wyraz dla tworzonego ciągu
#funkcja została napisana w jak najprostszy iteracyjny sposób
def choose_next_word_full_iterating(problem_properties):
    #print("avaible words przed: " + str(problem_properties.avaible_words_number))
    #wybranie odpowiednich wektorów z macierzy, które przechowują dane dla ostatnio wybranego słowa
    common_nucleotides_vector = problem_properties.common_nucleotides_matrix[problem_properties.current_word]
    next_word_power_vector = problem_properties.next_word_power_matrix[problem_properties.current_word]
    
    #jeśli obecna długość wynikowego ciągu zwiększona o pełną długość kolejnego słowa przekracza ograniczającą długość ciągu znaków,
    #to trzeba wykluczyć możliwość wyboru kolejnego słowa, które by to za bardzo zwiększyła długość wynikowej sekwencji
    #print(str(problem_properties.current_length) + " + " + str(problem_properties.word_len) + " > " + str(problem_properties.org_seq_len))
    if(problem_properties.current_length + problem_properties.word_len > problem_properties.org_seq_len):
        for j in range(1, problem_properties.words_num + 1):
            if((problem_properties.avaible_words[j]) and (problem_properties.current_length + (problem_properties.word_len - common_nucleotides_vector[j]) > problem_properties.org_seq_len)):
                problem_properties.avaible_words[j] = False
                problem_properties.avaible_words_number -= 1
        
        #print("Juz prawie koniec - mozna dopasowac jeszcze " + str(problem_properties.avaible_words_number) + " slow")
    
    #nie ma już dostępnych słów do doklejenia
    if(not(problem_properties.avaible_words_number)):
        return -1
    
    #obliczenie sumy mocy prawdopodobieństwa wyboru dla dostępnych słów
    sum_of_probability = 0
    for j in range(1, problem_properties.words_num + 1):
        if(problem_properties.avaible_words[j]):
            sum_of_probability += next_word_power_vector[j]

            
    """#print(str(max_sum) + " < " + str(sum_of_probability))
    if(max_sum < sum_of_probability):
        max_sum = sum_of_probability
        print("sum_of_probability = " + str(sum_of_probability))"""
    
    #realizacja wyboru zgodnie z gęstością prawdopodobieństwa - wybór punktu na odcinku złożonego z sumy mocy prawdopodobieństwa dostępnych połączeń
    point_in_range_of_sum_of_probability = random.uniform(0, sum_of_probability)
    
    #odnalezienie wybranego kolejnego słowa, które zostało wskazane przez wylosowany punkt na odcinku
    j = 1
    while(not(problem_properties.avaible_words[j])):
        j += 1
    
    current_sum_of_probability = next_word_power_vector[j]
    while(point_in_range_of_sum_of_probability > current_sum_of_probability):
        j += 1
        while(not(problem_properties.avaible_words[j])):
            j += 1
        current_sum_of_probability += next_word_power_vector[j]
    
    #print("avaible words po: " + str(problem_properties.avaible_words_number))
    
    return j

#funkcja wybiera kolejny wyraz dla tworzonego ciągu
#zwektoryzowana funkcja korzystająca z np.random.choice
def choose_next_word_with_numpy_tricks_1(problem_properties):
    #print("avaible words przed: " + str(problem_properties.avaible_words_number))
    #wybranie odpowiednich wektorów z macierzy, które przechowują dane dla ostatnio wybranego słowa
    common_nucleotides_vector = problem_properties.common_nucleotides_matrix[problem_properties.current_word]
    next_word_power_vector = problem_properties.next_word_power_matrix[problem_properties.current_word]
    
    #jeśli obecna długość wynikowego ciągu zwiększona o pełną długość kolejnego słowa przekracza ograniczającą długość ciągu znaków,
    #to trzeba wykluczyć możliwość wyboru kolejnego słowa, które by to za bardzo zwiększyła długość wynikowej sekwencji
    #print(str(problem_properties.current_length) + " + " + str(problem_properties.word_len) + " > " + str(problem_properties.org_seq_len))
    if(problem_properties.current_length + problem_properties.word_len > problem_properties.org_seq_len):
        mask = problem_properties.avaible_words * (problem_properties.current_length + problem_properties.word_len - common_nucleotides_vector > problem_properties.org_seq_len)
        problem_properties.avaible_words[mask] = False
        problem_properties.avaible_words_number -= np.sum(mask)
        
        #print("Juz prawie koniec - mozna dopasowac jeszcze " + str(problem_properties.avaible_words_number) + " slow")
    
    #nie ma już dostępnych słów do doklejenia
    if(not(problem_properties.avaible_words_number)):
        return -1
    
    #realizacja wyboru zgodnie z gęstością prawdopodobieństwa
    indices = np.arange(problem_properties.words_num + 1)
    p_active = next_word_power_vector[ problem_properties.avaible_words ]
    p_sum = np.sum(p_active)
    p_active = p_active / p_sum
    index = np.random.choice(indices[ problem_properties.avaible_words ], 1, p=p_active)
    
    #print("avaible words po: " + str(problem_properties.avaible_words_number))
    
    return index[0]

#funkcja wybiera kolejny wyraz dla tworzonego ciągu
#zwektoryzowana funkcja odpowiadająca idei wersji iteracyjnej
def choose_next_word_with_numpy_tricks_2(problem_properties):
    #print("avaible words przed: " + str(problem_properties.avaible_words_number))
    #wybranie odpowiednich wektorów z macierzy, które przechowują dane dla ostatnio wybranego słowa
    common_nucleotides_vector = problem_properties.common_nucleotides_matrix[problem_properties.current_word]
    next_word_power_vector = problem_properties.next_word_power_matrix[problem_properties.current_word]
    
    #jeśli obecna długość wynikowego ciągu zwiększona o pełną długość kolejnego słowa przekracza ograniczającą długość ciągu znaków,
    #to trzeba wykluczyć możliwość wyboru kolejnego słowa, które by to za bardzo zwiększyła długość wynikowej sekwencji
    #print(str(problem_properties.current_length) + " + " + str(problem_properties.word_len) + " > " + str(problem_properties.org_seq_len))
    if(problem_properties.current_length + problem_properties.word_len > problem_properties.org_seq_len):
        mask = problem_properties.avaible_words * (problem_properties.current_length + problem_properties.word_len - common_nucleotides_vector > problem_properties.org_seq_len)
        problem_properties.avaible_words[mask] = False
        problem_properties.avaible_words_number -= np.sum(mask)
        
        #print("Juz prawie koniec - mozna dopasowac jeszcze " + str(problem_properties.avaible_words_number) + " slow")
    
    #nie ma już dostępnych słów do doklejenia
    if(not(problem_properties.avaible_words_number)):
        return -1
    
    #wydzielenie potrzebnych danych o dostępnych do wyboru słowach
    next_word_power_not_used = next_word_power_vector[ problem_properties.avaible_words ]
    indices = np.arange(problem_properties.words_num + 1)[ problem_properties.avaible_words ]
    
    #obliczenie sumy mocy prawdopodobieństwa wyboru dla dostępnych słów
    sum_of_probability = np.sum(next_word_power_not_used)
    
    #realizacja wyboru zgodnie z gęstością prawdopodobieństwa - wybór punktu na odcinku złożonego z sumy mocy prawdopodobieństwa dostępnych połączeń
    point_in_range_of_sum_of_probability = random.uniform(0, sum_of_probability)
    
    #odnalezienie wybranego kolejnego słowa, które zostało wskazane przez wylosowany punkt na odcinku
    j = 0
    current_sum_of_probability = next_word_power_not_used[j]
    while(point_in_range_of_sum_of_probability > current_sum_of_probability):
        j += 1
        current_sum_of_probability += next_word_power_not_used[j]
    
    return indices[j]

#funkcja wybiera kolejny wyraz dla tworzonego ciągu
#zwektoryzowana funkcja korzystająca z bisekcji
def choose_next_word_with_numpy_tricks_3(problem_properties):
    #wybranie odpowiednich wektorów z macierzy, które przechowują dane dla ostatnio wybranego słowa
    common_nucleotides_vector = problem_properties.common_nucleotides_matrix[problem_properties.current_word]
    next_word_power_vector = problem_properties.next_word_power_matrix[problem_properties.current_word]
    
    #jeśli obecna długość wynikowego ciągu zwiększona o pełną długość kolejnego słowa przekracza ograniczającą długość ciągu znaków,
    #to trzeba wykluczyć możliwość wyboru kolejnego słowa, które by to za bardzo zwiększyła długość wynikowej sekwencji
    #print(str(problem_properties.current_length) + " + " + str(problem_properties.word_len) + " > " + str(problem_properties.org_seq_len))
    if(problem_properties.current_length + problem_properties.word_len > problem_properties.org_seq_len):
        mask = problem_properties.avaible_words * (problem_properties.current_length + problem_properties.word_len - common_nucleotides_vector > problem_properties.org_seq_len)
        problem_properties.avaible_words[mask] = False
        problem_properties.avaible_words_number -= np.sum(mask)
        
        #print("Juz prawie koniec - mozna dopasowac jeszcze " + str(problem_properties.avaible_words_number) + " slow")
    
    #nie ma już dostępnych słów do doklejenia
    if(not(problem_properties.avaible_words_number)):
        return -1
    
    #wydzielenie potrzebnych danych o dostępnych do wyboru słowach
    next_word_power_not_used = next_word_power_vector[ problem_properties.avaible_words ]
    indices = np.arange(problem_properties.words_num + 1)[ problem_properties.avaible_words ]
    
    #obliczenie sumy mocy prawdopodobieństwa wyboru dla dostępnych słów
    sum_of_probability = np.sum(next_word_power_not_used)
    
    #realizacja wyboru zgodnie z gęstością prawdopodobieństwa - wybór punktu na odcinku złożonego z sumy mocy prawdopodobieństwa dostępnych połączeń
    point_in_range_of_sum_of_probability = random.uniform(0, sum_of_probability)    
    
    #odnalezienie wybranego kolejnego słowa, które zostało wskazane przez wylosowany punkt na odcinku
    P = 0
    K = next_word_power_not_used.size - 1
    while(P != K):
        S = (P + K) // 2
        if(np.sum(next_word_power_not_used[:S+1]) < point_in_range_of_sum_of_probability):
            P = S + 1
        else:
            K = S
    
    return indices[P]

#funkcja wybiera kolejny wyraz dla tworzonego ciągu
#zwektoryzowana funkcja korzystająca z np.cumsum
def choose_next_word_with_numpy_tricks_4(problem_properties):
    #wybranie odpowiednich wektorów z macierzy, które przechowują dane dla ostatnio wybranego słowa
    common_nucleotides_vector = problem_properties.common_nucleotides_matrix[problem_properties.current_word]
    next_word_power_vector = problem_properties.next_word_power_matrix[problem_properties.current_word]
    
    #jeśli obecna długość wynikowego ciągu zwiększona o pełną długość kolejnego słowa przekracza ograniczającą długość ciągu znaków,
    #to trzeba wykluczyć możliwość wyboru kolejnego słowa, które by to za bardzo zwiększyła długość wynikowej sekwencji
    #print(str(problem_properties.current_length) + " + " + str(problem_properties.word_len) + " > " + str(problem_properties.org_seq_len))
    if(problem_properties.current_length + problem_properties.word_len > problem_properties.org_seq_len):
        mask = problem_properties.avaible_words * (problem_properties.current_length + problem_properties.word_len - common_nucleotides_vector > problem_properties.org_seq_len)
        problem_properties.avaible_words[mask] = False
        problem_properties.avaible_words_number -= np.sum(mask)
        
        #print("Juz prawie koniec - mozna dopasowac jeszcze " + str(problem_properties.avaible_words_number) + " slow")
    
    #nie ma już dostępnych słów do doklejenia
    if(not(problem_properties.avaible_words_number)):
        return -1
    
    #wydzielenie potrzebnych danych o dostępnych do wyboru słowach
    next_word_power_not_used = next_word_power_vector[ problem_properties.avaible_words ]
    indices = np.arange(problem_properties.words_num + 1)[ problem_properties.avaible_words ]
    
    #wyznaczenie kumulacyjnej sumy z wektora wag prawdopodobieństw dostępnych wyrazów
    next_word_power_not_used_cumsum = np.zeros(next_word_power_not_used.shape)
    np.cumsum(next_word_power_not_used, out=next_word_power_not_used_cumsum)
    
    #realizacja wyboru zgodnie z gęstością prawdopodobieństwa - wybór punktu na odcinku złożonego z sumy mocy prawdopodobieństwa dostępnych połączeń
    point_in_range_of_sum_of_probability = random.uniform(0, next_word_power_not_used_cumsum[-1])
    
    #odnalezienie wybranego kolejnego słowa, które zostało wskazane przez wylosowany punkt na odcinku
    j = np.searchsorted(next_word_power_not_used_cumsum, point_in_range_of_sum_of_probability)
    
    return indices[j]





def create_sequence(problem_properties, choose_next_word_function):
    problem_properties.prepare_attempts_vars()

    problem_properties.avaible_words[0] = False        #odznaczenie użycia już zerowego słowa ('X')
    problem_properties.current_word = 0            #ustawienie startowego słowa -> słowo zerowe, czyli zaczynane jest tworzenie sekwencji
    problem_properties.sequence.append(problem_properties.current_word)

    #nieustannie wykonująca się pętla - zostanie przerwana, gdy nie będzie już możliwości doczepienia słowa przy ograniczeniu ilości znaków w wynikowym ciągu
    while(True):
        next_word = choose_next_word_function(problem_properties)
        #print("Wybrane slowo: " + str(next_word))

        if(next_word == -1):
            break

        problem_properties.current_length += problem_properties.word_len - problem_properties.common_nucleotides_matrix[problem_properties.current_word, next_word]
        problem_properties.avaible_words_number -= 1
        problem_properties.avaible_words[next_word] = False

        """print(problem_properties.words[problem_properties.current_word])
        print((problem_properties.word_len - problem_properties.common_nucleotides_matrix[problem_properties.current_word, next_word]) * ' ' + problem_properties.words[problem_properties.next_word] + "\tPokrywanie na tylu znakach: " + str(problem_properties.common_nucleotides_matrix[problem_properties.current_word, next_word]))
        print("Liczba niewykorzystanych jeszcze slow: " + str(problem_properties.avaible_words_number))
        print("Obecna dlugosc ciagu: " + str(problem_properties.current_length))
        print("\n")"""

        problem_properties.current_word = next_word
        problem_properties.sequence.append(problem_properties.current_word)

        
        
        
        
#funkcja dokonuje wygładzania wartości wag prawdopodobieństw -> zmienia szanse wyboru na bardziej równe, zachowująć jednocześnie porządek preferencji
def compensation_of_next_word_power_matrix_full_iterating(problem_properties):
    
    for i in range(problem_properties.words_num + 1):
        max_value = np.max(problem_properties.next_word_power_matrix[i])
        sum_value = np.sum(problem_properties.next_word_power_matrix[i])
        
        #jeśli maksymalna wartość zdecydowanie się wyróżnia, to należy złagodzić dysproporcje między wagami prawdopodobieństw
        #print("max_value = " + str(max_value))
        #print("sum_value * problem_properties.compensation_coef = " + str(sum_value * problem_properties.compensation_coef))
        if(max_value > sum_value * problem_properties.compensation_coef):
            #poszukiwanie minimalnej wartości wagi prawdopodobieństwa nie mniejszej niż wartość 1. Wartość ta jest potrzebna do wzoru wygładzającego
            min_value = max_value
            for j in range(problem_properties.words_num + 1):
                if((problem_properties.next_word_power_matrix[i, j] >= 1) and (min_value > problem_properties.next_word_power_matrix[i, j])):
                    min_value = problem_properties.next_word_power_matrix[i, j]
            
            #wygładzanie wag większych niż wartość 1
            for j in range(problem_properties.words_num + 1):
                if(min_value < problem_properties.next_word_power_matrix[i, j]):
                    problem_properties.next_word_power_matrix[i, j] = min_value * (1 + math.log(problem_properties.next_word_power_matrix[i, j] / min_value, problem_properties.base_of_logarithm))

#funkcja dokonuje wygładzania wartości wag prawdopodobieństw -> zmienia szanse wyboru na bardziej równe, zachowująć jednocześnie porządek preferencji
#operacja zmiany wag w danym wierszu została tutaj zwektoryzowana
def compensation_of_next_word_power_matrix_with_numpy_tricks_1(problem_properties):
    
    for i in range(problem_properties.words_num + 1):
        max_value = np.max(problem_properties.next_word_power_matrix[i])
        sum_value = np.sum(problem_properties.next_word_power_matrix[i])
        
        #jeśli maksymalna wartość zdecydowanie się wyróżnia, to należy złagodzić dysproporcje między wagami prawdopodobieństw
        #print("max_value = " + str(max_value))
        #print("sum_value * problem_properties.compensation_coef = " + str(sum_value * problem_properties.compensation_coef))
        if(max_value > sum_value * problem_properties.compensation_coef):
            #poszukiwanie minimalnej wartości wagi prawdopodobieństwa nie mniejszej niż wartość 1. Wartość ta jest potrzebna do wzoru wygładzającego
            min_value = min(problem_properties.next_word_power_matrix[i, problem_properties.next_word_power_matrix[i] >= 1])
            
            #wygładzanie wag większych niż wartość 1
            problem_properties.next_word_power_matrix[i, problem_properties.next_word_power_matrix[i] > min_value] = min_value * (1 + np.log(problem_properties.next_word_power_matrix[i, problem_properties.next_word_power_matrix[i] > min_value] / min_value) / np.log(problem_properties.base_of_logarithm))

#funkcja dokonuje wygładzania wartości wag prawdopodobieństw -> zmienia szanse wyboru na bardziej równe, zachowująć jednocześnie porządek preferencji
#cała procedura została zwektoryzowana
def compensation_of_next_word_power_matrix_with_numpy_tricks_2(problem_properties):
    
    max_value = np.max(problem_properties.next_word_power_matrix, axis=1)
    sum_value = np.sum(problem_properties.next_word_power_matrix, axis=1)
    
    #odwzorowanie maksymalnych wartości wierszy na całą macierz
    max_value_matrix = np.zeros((problem_properties.words_num + 1, problem_properties.words_num + 1))
    max_value_matrix[:] = max_value
    max_value_matrix = max_value_matrix.transpose()

    #odwzorowanie sumy wartości wierszy na całą macierz
    sum_value_matrix = np.zeros((problem_properties.words_num + 1, problem_properties.words_num + 1))
    sum_value_matrix[:] = sum_value
    sum_value_matrix = sum_value_matrix.transpose()

    #wskazanie wierszy w macierzy, w których trzeba wykonać wygładzanie
    mask_row = max_value_matrix > sum_value_matrix * problem_properties.compensation_coef

    #przygotowanie kopii next_word_power_matrix, którą będzie trzeba zmodyfikować w celu odnalezienia minimalnej wartości większej niż 1 dla każdego wiersza
    tmp_next_word_power_matrix = np.copy(problem_properties.next_word_power_matrix)
    mask_tmp = tmp_next_word_power_matrix < 1
    tmp_next_word_power_matrix[mask_tmp] = max_value_matrix[mask_tmp]

    #wyznaczenie i odwzorowanie na całą macierz minimalnej wartości większej niż 1 dla każdego wiersza
    min_value = np.min(tmp_next_word_power_matrix, axis=1)
    min_value_matrix = np.zeros((problem_properties.words_num + 1, problem_properties.words_num + 1))
    min_value_matrix[:] = min_value
    min_value_matrix = min_value_matrix.transpose()

    #wskazanie elementów, które należy zmodyfikować
    mask_final = ((problem_properties.next_word_power_matrix >= min_value_matrix) * (mask_row))

    #wykonanie faktycznego wygładzania dominujących wartości w problem_properties.next_word_power_matrix
    min_value_vector = min_value_matrix[mask_final]
    problem_properties.next_word_power_matrix[mask_final] = min_value_vector * (1 + np.log(problem_properties.next_word_power_matrix[mask_final] / min_value_vector) / np.log(problem_properties.base_of_logarithm))
    
    
    
    
    
#funkcja zwraca długość utworzonej sekwencji na podstawie indeksów wyrazów i macierzy zawierającej długości wspólnych sufiksów-prefiksów wszystkich par słów.
def length_of_created_sequence_full_iterating(sequence, common_nucleotides_matrix, word_len):
    result = 0
    for i in range(len(sequence) - 1):
        result += word_len - common_nucleotides_matrix[sequence[i], sequence[i + 1]]
    
    return result


#funkcja zwraca długość utworzonej sekwencji na podstawie indeksów wyrazów i macierzy zawierającej długości wspólnych sufiksów-prefiksów wszystkich par słów.
#przyspieszenie przetwarzania poprzez wektoryzację obliczeń
def length_of_created_sequence_with_numpy_tricks(sequence, common_nucleotides_matrix, word_len):
    first_word = sequence[:-1]
    second_word = sequence[1:]
    
    result = np.sum(word_len - common_nucleotides_matrix[first_word, second_word])
    return result
    
    
    
    
    
#funkcja nagradza najlepsze rozwiązania przeprowadzając odpowiednie modyfikacje w macierzy wag prawdopodobieństw
def reward_the_best_solutions_full_iterating(problem_properties):
    
    """for i in range(len(problem_properties.sequences)):
        print(str(i) + ":\t" + str(len(problem_properties.sequences[i])))"""
    
    problem_properties.sequences.sort(key = lambda x: length_of_created_sequence_with_numpy_tricks(x, problem_properties.common_nucleotides_matrix, problem_properties.word_len), reverse=False)
    problem_properties.sequences.sort(key = lambda x: len(x), reverse=True)

    """for i in range(len(problem_properties.sequences)):
        print(str(i) + ":\t" + str(len(problem_properties.sequences[i])))"""

    #nagrodzenie najlepszych rozwiązań
    amount_of_the_best_sequences = int(problem_properties.attempts_number * problem_properties.part_of_the_best_sequences)
    """for i in range(amount_of_the_best_sequences):
        print(str(i) + ":\t" + str(len(problem_properties.sequences[i])))"""
    for iSequence in range(amount_of_the_best_sequences):
        sequence = problem_properties.sequences[iSequence]
        price_for_sequence = 1 + (problem_properties.best_price_for_sequence - 1) * ((amount_of_the_best_sequences - iSequence) / amount_of_the_best_sequences)
        #print("price_for_sequence = " + str(price_for_sequence))
        for iConnection in range(len(sequence) - 1):
            first_word = sequence[iConnection]
            second_word = sequence[iConnection + 1]

            price_for_connection = 1 + (price_for_sequence - 1) * (problem_properties.common_nucleotides_matrix[first_word, second_word] / (problem_properties.word_len - 1))
            problem_properties.next_word_power_matrix[first_word, second_word] *= price_for_connection


        """wyswietlanie mnożnika dla połączeń - im bardziej wyrazy się zazębiają, tym większa liniowo jest wartość mnożnika
        for i in range(problem_properties.word_len):
            price_for_connection = 1 + (price_for_sequence - 1) * (i / (problem_properties.word_len - 1))
            print("price_for_connection = " + str(price_for_connection))"""
        
#funkcja nagradza najlepsze rozwiązania przeprowadzając odpowiednie modyfikacje w macierzy wag prawdopodobieństw
#przyspieszenie przetwarzania poprzez wektoryzację obliczeń
def reward_the_best_solutions_with_numpy_tricks(problem_properties):

    problem_properties.sequences.sort(key = lambda x: length_of_created_sequence_with_numpy_tricks(x, problem_properties.common_nucleotides_matrix, problem_properties.word_len), reverse=False)
    problem_properties.sequences.sort(key = lambda x: len(x), reverse=True)

    #nagrodzenie najlepszych rozwiązań
    amount_of_the_best_sequences = int(problem_properties.attempts_number * problem_properties.part_of_the_best_sequences)
    for iSequence in range(amount_of_the_best_sequences):
        sequence = np.array(problem_properties.sequences[iSequence])
        price_for_sequence = 1 + (problem_properties.best_price_for_sequence - 1) * ((amount_of_the_best_sequences - iSequence) / amount_of_the_best_sequences)
        #print("price_for_sequence = " + str(price_for_sequence))
        
        first_word = sequence[:-1]
        second_word = sequence[1:]
        price_for_connection = 1 + (price_for_sequence - 1) * (problem_properties.common_nucleotides_matrix[first_word, second_word] / (problem_properties.word_len - 1))
        problem_properties.next_word_power_matrix[first_word, second_word] *= price_for_connection

        
        
        
        
#funkcja wykonuje zapisanie długości otrzymanych sekwencji do listy sequences_lengths_of_all_iterations
def remember_sequences_lengths_full_iterating(problem_properties):
    
    result = np.zeros(problem_properties.attempts_number, dtype=int)
    
    for i in range(problem_properties.attempts_number):
        result[i] = len(problem_properties.sequences[i])
    
    problem_properties.sequences_lengths_of_all_iterations.append(result)
    

        
#funkcja wykonuje zapisanie długości otrzymanych sekwencji do listy sequences_lengths_of_all_iterations
#przyspieszenie przetwarzania poprzez wektoryzację obliczeń
def remember_sequences_lengths_with_numpy_tricks(problem_properties):
    
    result = np.array(list(map(len, problem_properties.sequences)))
    
    problem_properties.sequences_lengths_of_all_iterations.append(result)
        
        
        
        
        
def run_ant_algorithm_constant_iteration_number_with_message(problem_properties, filename, method_of_probability_assignment=2):
    random.seed(0)
    with open(filename, 'r') as file:
        inst_name = filename[filename.rfind('/') + 1:]

        words = file.read().splitlines()

        word_len = len(words[0])
        org_words_num = int(re.search('(?<=\.)[0-9]+', inst_name).group(0))
        org_seq_len = org_words_num + word_len - 1
        neg_errors_number = re.search('(?<=\-)[0-9]+', inst_name)
        neg_errors_number = int(neg_errors_number.group(0)) if neg_errors_number else 0
        pos_errors_number = re.search('(?<=\+)[0-9]+', inst_name)
        pos_errors_number = int(pos_errors_number.group(0)) if pos_errors_number else 0
        words_num = len(words)

        print("nazwa pliku: " + inst_name)
        print("długość słowa: " + str(word_len))
        print("oryginalna liczba słów: " + str(org_words_num))
        print("oryginalna długość sekwencji: "+ str(org_seq_len))
        print("liczba błędów negatywnych: " + str(neg_errors_number))
        print("liczba błędów pozytywnych: " + str(pos_errors_number))
        print("liczba oligonukleotydów: " + str(words_num))

        #dodanie pozornego słowa w celu zaimplementowania sytuacji wyboru pierwszego wyrazu rozpoczynającego sekwencję
        words = ["Poczatek"] + words
        print(words[:5])
        print("\n")


        #zapisanie ważnych informacji w obiekcie klasy ProblemPropertiesClass
        problem_properties.set_instance_info(words, word_len, org_seq_len, words_num)


        #wyznaczenie macierzy mówiącej o długości wspólnego i-tego sufiksu z j-tym prefiksem
        print("common_nucleotides_matrix()")
        generate_common_nucleotides_matrix_full_iterating(problem_properties)
        #print(common_nucleotides_matrix[0, :])

        print("set_starting_wages()")
        problem_properties.wages = set_starting_wages(problem_properties.word_len, problem_properties.k_set_starting_wages)
        #print(wages)


        #wyznaczanie macierzy mocy prawdopodobieństwa następienia j-tego wyrazu po i-tym
        if(method_of_probability_assignment == 1):
            print("generate_next_word_power_matrix_1_with_numpy_tricks()")
            generate_next_word_power_matrix_1_with_numpy_tricks(problem_properties)
        else:
            print("generate_next_word_power_matrix_2_with_numpy_tricks_2()")
            generate_next_word_power_matrix_2_with_numpy_tricks_2(problem_properties)

        #ustawienie wag prawdopodobieństwa dla rozpoczęcia tworzenia sekwencji przez dany wyraz
        if(method_of_probability_assignment == 1):
            print("generate_chance_for_first_word_1_with_numpy_tricks()")
            generate_chance_for_first_word_1_with_numpy_tricks(problem_properties)
        else:
            print("generate_chance_for_first_word_2_with_numpy_tricks()")
            generate_chance_for_first_word_2_with_numpy_tricks(problem_properties)

        #wykonywanie poszczególnej iteracji; wykonywanie pętli tyle razy, ile jest zadeklarowanych iteracji do wykonania
        for indeks_of_iteration in range(problem_properties.iterations_number):
            problem_properties.prepare_iterations_vars()


            #tworzenie wynikowej sekwencji; powtarzanie prób uzyskania rozwiązania tyle razy, ile jest to określone parametrem 'problem_properties.attempts_number'
            print("\nIteracja " + str(indeks_of_iteration + 1) + ":\t\nTrwa wykonywanie wszystkich " + str(problem_properties.attempts_number) + " prób utworzenia wynikowej sekwencji.")
            for indeks_of_attempt in range(problem_properties.attempts_number):
                #utworzenie sekwencji
                create_sequence(problem_properties, choose_next_word_with_numpy_tricks_4)

                #zapamiętanie uzyskanej sekwencji
                problem_properties.sequences.append(problem_properties.sequence)


            print("Wykonano wszystkie sekwencje w obecnej iteracji")


            #nagrodzenie najlepszych rozwiązań przeprowadzając odpowiednie modyfikacje w macierzy wag prawdopodobieństw
            print("reward_the_best_solutions_full_iterating()")
            reward_the_best_solutions_with_numpy_tricks(problem_properties)
            
            #zapamiętanie ewentualnego lepszego rozwiązania składającego się z większej ilości wyrazów
            tmp_len = len(problem_properties.sequences[0])
            if(problem_properties.best_sequence_len < tmp_len):
                problem_properties.best_sequence_len = tmp_len
                problem_properties.best_sequence = problem_properties.sequences[0]
                problem_properties.iteration_with_the_best_sequence = indeks_of_iteration + 1


            print(problem_properties.sequences[0])
            for i in range(10):
                znaki = length_of_created_sequence_with_numpy_tricks(problem_properties.sequences[i], problem_properties.common_nucleotides_matrix, problem_properties.word_len)
                print(str(i) + ":\twyrazy: " + str(len(problem_properties.sequences[i]) - 1) + "\tznaki: " + str(znaki))
            
            
            #wywołanie funkcji zapamiętującej długości wszystkich uzyskanych sekwencji w danej iteracji
            print("remember_sequences_lengths_with_numpy_tricks()")
            remember_sequences_lengths_with_numpy_tricks(problem_properties)
            

            #wygładzanie macierzy wag prawdopodobieństwa w celu zapobiegnięcia nadmiernemu zdominowaniu przez najlepsze rozwiązanie
            print("compensation_of_next_word_power_matrix()")
            compensation_of_next_word_power_matrix_with_numpy_tricks_2(problem_properties)

        print("koniec")
    


def run_ant_algorithm_constant_iteration_number(problem_properties, filename, method_of_probability_assignment=2):
    random.seed(0)
    with open(filename, 'r') as file:
        inst_name = filename[filename.rfind('/') + 1:]

        words = file.read().splitlines()

        word_len = len(words[0])
        org_words_num = int(re.search('(?<=\.)[0-9]+', inst_name).group(0))
        org_seq_len = org_words_num + word_len - 1
        neg_errors_number = re.search('(?<=\-)[0-9]+', inst_name)
        neg_errors_number = int(neg_errors_number.group(0)) if neg_errors_number else 0
        pos_errors_number = re.search('(?<=\+)[0-9]+', inst_name)
        pos_errors_number = int(pos_errors_number.group(0)) if pos_errors_number else 0
        words_num = len(words)

        #dodanie pozornego słowa w celu zaimplementowania sytuacji wyboru pierwszego wyrazu rozpoczynającego sekwencję
        words = ["Poczatek"] + words


        #zapisanie ważnych informacji w obiekcie klasy ProblemPropertiesClass
        problem_properties.set_instance_info(words, word_len, org_seq_len, words_num)


        #wyznaczenie macierzy mówiącej o długości wspólnego i-tego sufiksu z j-tym prefiksem
        generate_common_nucleotides_matrix_full_iterating(problem_properties)
        #print(common_nucleotides_matrix[0, :])

        problem_properties.wages = set_starting_wages(problem_properties.word_len, problem_properties.k_set_starting_wages)
        #print(wages)


        #wyznaczanie macierzy mocy prawdopodobieństwa następienia j-tego wyrazu po i-tym
        if(method_of_probability_assignment == 1):
            generate_next_word_power_matrix_1_with_numpy_tricks(problem_properties)
        else:
            generate_next_word_power_matrix_2_with_numpy_tricks_2(problem_properties)

        #ustawienie wag prawdopodobieństwa dla rozpoczęcia tworzenia sekwencji przez dany wyraz
        if(method_of_probability_assignment == 1):
            generate_chance_for_first_word_1_with_numpy_tricks(problem_properties)
        else:
            generate_chance_for_first_word_2_with_numpy_tricks(problem_properties)

        #wykonywanie poszczególnej iteracji; wykonywanie pętli tyle razy, ile jest zadeklarowanych iteracji do wykonania
        for indeks_of_iteration in range(problem_properties.iterations_number):
            problem_properties.prepare_iterations_vars()


            #tworzenie wynikowej sekwencji; powtarzanie prób uzyskania rozwiązania tyle razy, ile jest to określone parametrem 'problem_properties.attempts_number'
            for indeks_of_attempt in range(problem_properties.attempts_number):
                #utworzenie sekwencji
                create_sequence(problem_properties, choose_next_word_with_numpy_tricks_4)

                #zapamiętanie uzyskanej sekwencji
                problem_properties.sequences.append(problem_properties.sequence)
                


            #nagrodzenie najlepszych rozwiązań przeprowadzając odpowiednie modyfikacje w macierzy wag prawdopodobieństw
            reward_the_best_solutions_with_numpy_tricks(problem_properties)
            
            #zapamiętanie ewentualnego lepszego rozwiązania składającego się z większej ilości wyrazów
            tmp_len = len(problem_properties.sequences[0])
            if(problem_properties.best_sequence_len < tmp_len):
                problem_properties.best_sequence_len = tmp_len
                problem_properties.best_sequence = problem_properties.sequences[0]
                problem_properties.iteration_with_the_best_sequence = indeks_of_iteration + 1

                
            
            #wywołanie funkcji zapamiętującej długości wszystkich uzyskanych sekwencji w danej iteracji
            remember_sequences_lengths_with_numpy_tricks(problem_properties)
            

            #wygładzanie macierzy wag prawdopodobieństwa w celu zapobiegnięcia nadmiernemu zdominowaniu przez najlepsze rozwiązanie
            compensation_of_next_word_power_matrix_with_numpy_tricks_2(problem_properties)



def run_ant_algorithm_with_message(problem_properties, filename, method_of_probability_assignment=2):
    random.seed(0)
    with open(filename, 'r') as file:
        inst_name = filename[filename.rfind('/') + 1:]

        words = file.read().splitlines()

        word_len = len(words[0])
        org_words_num = int(re.search('(?<=\.)[0-9]+', inst_name).group(0))
        org_seq_len = org_words_num + word_len - 1
        neg_errors_number = re.search('(?<=\-)[0-9]+', inst_name)
        neg_errors_number = int(neg_errors_number.group(0)) if neg_errors_number else 0
        pos_errors_number = re.search('(?<=\+)[0-9]+', inst_name)
        pos_errors_number = int(pos_errors_number.group(0)) if pos_errors_number else 0
        words_num = len(words)

        print("nazwa pliku: " + inst_name)
        print("długość słowa: " + str(word_len))
        print("oryginalna liczba słów: " + str(org_words_num))
        print("oryginalna długość sekwencji: "+ str(org_seq_len))
        print("liczba błędów negatywnych: " + str(neg_errors_number))
        print("liczba błędów pozytywnych: " + str(pos_errors_number))
        print("liczba oligonukleotydów: " + str(words_num))

        #dodanie pozornego słowa w celu zaimplementowania sytuacji wyboru pierwszego wyrazu rozpoczynającego sekwencję
        words = ["Poczatek"] + words
        print(words[:5])
        print("\n")


        #zapisanie ważnych informacji w obiekcie klasy ProblemPropertiesClass
        problem_properties.set_instance_info(words, word_len, org_seq_len, words_num)


        #wyznaczenie macierzy mówiącej o długości wspólnego i-tego sufiksu z j-tym prefiksem
        print("common_nucleotides_matrix()")
        generate_common_nucleotides_matrix_full_iterating(problem_properties)
        #print(common_nucleotides_matrix[0, :])

        print("set_starting_wages()")
        problem_properties.wages = set_starting_wages(problem_properties.word_len, problem_properties.k_set_starting_wages)
        #print(wages)


        #wyznaczanie macierzy mocy prawdopodobieństwa następienia j-tego wyrazu po i-tym
        if(method_of_probability_assignment == 1):
            print("generate_next_word_power_matrix_1_with_numpy_tricks()")
            generate_next_word_power_matrix_1_with_numpy_tricks(problem_properties)
        else:
            print("generate_next_word_power_matrix_2_with_numpy_tricks_2()")
            generate_next_word_power_matrix_2_with_numpy_tricks_2(problem_properties)

        #ustawienie wag prawdopodobieństwa dla rozpoczęcia tworzenia sekwencji przez dany wyraz
        if(method_of_probability_assignment == 1):
            print("generate_chance_for_first_word_1_with_numpy_tricks()")
            generate_chance_for_first_word_1_with_numpy_tricks(problem_properties)
        else:
            print("generate_chance_for_first_word_2_with_numpy_tricks()")
            generate_chance_for_first_word_2_with_numpy_tricks(problem_properties)

        #wykonywanie poszczególnej iteracji; wykonywanie pętli, dopóki po określonej liczbie iteracji nie znajdzie się lepsze rozwiązanie
        problem_properties.iterations_number = 0
        while(problem_properties.iterations_number - problem_properties.iteration_with_the_best_sequence < problem_properties.iterations_number_limit):
            problem_properties.prepare_iterations_vars()
            problem_properties.iterations_number += 1


            #tworzenie wynikowej sekwencji; powtarzanie prób uzyskania rozwiązania tyle razy, ile jest to określone parametrem 'problem_properties.attempts_number'
            print("\nIteracja " + str(problem_properties.iterations_number) + ":\t\nTrwa wykonywanie wszystkich " + str(problem_properties.attempts_number) + " prób utworzenia wynikowej sekwencji.")
            for indeks_of_attempt in range(problem_properties.attempts_number):
                #utworzenie sekwencji
                create_sequence(problem_properties, choose_next_word_with_numpy_tricks_4)

                #zapamiętanie uzyskanej sekwencji
                problem_properties.sequences.append(problem_properties.sequence)


            print("Wykonano wszystkie sekwencje w obecnej iteracji")


            #nagrodzenie najlepszych rozwiązań przeprowadzając odpowiednie modyfikacje w macierzy wag prawdopodobieństw
            print("reward_the_best_solutions_full_iterating()")
            reward_the_best_solutions_with_numpy_tricks(problem_properties)
            
            #zapamiętanie ewentualnego lepszego rozwiązania składającego się z większej ilości wyrazów
            tmp_len = len(problem_properties.sequences[0])
            if(problem_properties.best_sequence_len < tmp_len):
                problem_properties.best_sequence_len = tmp_len
                problem_properties.best_sequence = problem_properties.sequences[0]
                problem_properties.iteration_with_the_best_sequence = problem_properties.iterations_number


            print(problem_properties.sequences[0])
            for i in range(10):
                znaki = length_of_created_sequence_with_numpy_tricks(problem_properties.sequences[i], problem_properties.common_nucleotides_matrix, problem_properties.word_len)
                print(str(i) + ":\twyrazy: " + str(len(problem_properties.sequences[i]) - 1) + "\tznaki: " + str(znaki))

                
            
            #wywołanie funkcji zapamiętującej długości wszystkich uzyskanych sekwencji w danej iteracji
            print("remember_sequences_lengths_with_numpy_tricks()")
            remember_sequences_lengths_with_numpy_tricks(problem_properties)

            
            #wygładzanie macierzy wag prawdopodobieństwa w celu zapobiegnięcia nadmiernemu zdominowaniu przez najlepsze rozwiązanie
            print("compensation_of_next_word_power_matrix()")
            compensation_of_next_word_power_matrix_with_numpy_tricks_2(problem_properties)

        print("koniec")
    


def run_ant_algorithm(problem_properties, filename, method_of_probability_assignment=2):
    random.seed(0)
    with open(filename, 'r') as file:
        inst_name = filename[filename.rfind('/') + 1:]

        words = file.read().splitlines()

        word_len = len(words[0])
        org_words_num = int(re.search('(?<=\.)[0-9]+', inst_name).group(0))
        org_seq_len = org_words_num + word_len - 1
        neg_errors_number = re.search('(?<=\-)[0-9]+', inst_name)
        neg_errors_number = int(neg_errors_number.group(0)) if neg_errors_number else 0
        pos_errors_number = re.search('(?<=\+)[0-9]+', inst_name)
        pos_errors_number = int(pos_errors_number.group(0)) if pos_errors_number else 0
        words_num = len(words)

        #dodanie pozornego słowa w celu zaimplementowania sytuacji wyboru pierwszego wyrazu rozpoczynającego sekwencję
        words = ["Poczatek"] + words


        #zapisanie ważnych informacji w obiekcie klasy ProblemPropertiesClass
        problem_properties.set_instance_info(words, word_len, org_seq_len, words_num)


        #wyznaczenie macierzy mówiącej o długości wspólnego i-tego sufiksu z j-tym prefiksem
        generate_common_nucleotides_matrix_full_iterating(problem_properties)

        problem_properties.wages = set_starting_wages(problem_properties.word_len, problem_properties.k_set_starting_wages)


        #wyznaczanie macierzy mocy prawdopodobieństwa następienia j-tego wyrazu po i-tym
        if(method_of_probability_assignment == 1):
            generate_next_word_power_matrix_1_with_numpy_tricks(problem_properties)
        else:
            generate_next_word_power_matrix_2_with_numpy_tricks_2(problem_properties)

        #ustawienie wag prawdopodobieństwa dla rozpoczęcia tworzenia sekwencji przez dany wyraz
        if(method_of_probability_assignment == 1):
            generate_chance_for_first_word_1_with_numpy_tricks(problem_properties)
        else:
            generate_chance_for_first_word_2_with_numpy_tricks(problem_properties)

        #wykonywanie poszczególnej iteracji; wykonywanie pętli, dopóki po określonej liczbie iteracji nie znajdzie się lepsze rozwiązanie
        problem_properties.iterations_number = 0
        while(problem_properties.iterations_number - problem_properties.iteration_with_the_best_sequence < problem_properties.iterations_number_limit):
            problem_properties.prepare_iterations_vars()
            problem_properties.iterations_number += 1


            #tworzenie wynikowej sekwencji; powtarzanie prób uzyskania rozwiązania tyle razy, ile jest to określone parametrem 'problem_properties.attempts_number'
            for indeks_of_attempt in range(problem_properties.attempts_number):
                #utworzenie sekwencji
                create_sequence(problem_properties, choose_next_word_with_numpy_tricks_4)

                #zapamiętanie uzyskanej sekwencji
                problem_properties.sequences.append(problem_properties.sequence)
                


            #nagrodzenie najlepszych rozwiązań przeprowadzając odpowiednie modyfikacje w macierzy wag prawdopodobieństw
            reward_the_best_solutions_with_numpy_tricks(problem_properties)
            
            #zapamiętanie ewentualnego lepszego rozwiązania składającego się z większej ilości wyrazów
            tmp_len = len(problem_properties.sequences[0])
            if(problem_properties.best_sequence_len < tmp_len):
                problem_properties.best_sequence_len = tmp_len
                problem_properties.best_sequence = problem_properties.sequences[0]
                problem_properties.iteration_with_the_best_sequence = problem_properties.iterations_number

                    
            #wywołanie funkcji zapamiętującej długości wszystkich uzyskanych sekwencji w danej iteracji
            remember_sequences_lengths_with_numpy_tricks(problem_properties)
            

            #wygładzanie macierzy wag prawdopodobieństwa w celu zapobiegnięcia nadmiernemu zdominowaniu przez najlepsze rozwiązanie
            compensation_of_next_word_power_matrix_with_numpy_tricks_2(problem_properties)

<hr/>
<h1>Wywołanie algorytmu mrówkowego dla danego pliku i danych parametrów.</h1>

In [ ]:
problem_properties = ProblemPropertiesClass()

#W konstruktorze klasy ProblemPropertiesClass zostały ustawione domyślne parametry, lecz można je również swobodnie zmieniać
#problem_properties.iterations_number = 5
#problem_properties.attempts_number = 10
#problem_properties.part_of_the_best_sequences = 0.5
#problem_properties.best_price_for_sequence = 3
#problem_properties.k_set_starting_wages = 5
#problem_properties.compensation_coef = 0.2
#problem_properties.base_of_logarithm = 20

filename = negative_random_filenames[2]
#filename = negative_repetitions_filenames[0]
#filename = positive_end_errors_filenames[0]
#filename = positive_random_filenames[2]

#run_ant_algorithm_constant_iteration_number_with_message(problem_properties, filename, method_of_probability_assignment=2)
#run_ant_algorithm_constant_iteration_number(problem_properties, filename, method_of_probability_assignment=2)
run_ant_algorithm_with_message(problem_properties, filename, method_of_probability_assignment=2)
#run_ant_algorithm(problem_properties, filename, method_of_probability_assignment=2)

print("Najlepsze rozwiązanie zostało złożone z " + str(problem_properties.best_sequence_len - 1) + " wyrazów.")
print("Znalezione zostało w " + str(problem_properties.iteration_with_the_best_sequence) + ". iteracji.")
print("Ogółem wykonano "+ str(problem_properties.iterations_number) + " iteracji.")
print(problem_properties.best_sequence[1:])

<hr/>
<h3>Próbna parametryzacja dla małego pliku w celu zrozumienia procesu</h3>

In [ ]:
k_set_starting_wages_values = [1, 2, 3, 4, 5, 7, 9, 12, 15, 20]
attempts_to_make_average = 3
filename = 'instances\\positive_random_custom\\1.20+10'
print("Plik: " + str(filename))

number_of_parameters_values = len(k_set_starting_wages_values)
for indeks_of_parameter in range(number_of_parameters_values):
    k_set_starting_wages_value = k_set_starting_wages_values[indeks_of_parameter]
    
    print("k_set_starting_wages_value = " + str(k_set_starting_wages_value))
    for attempt in range(attempts_to_make_average):
        print("attempt = " + str(attempt))
        problem_properties = ProblemPropertiesClass()
        problem_properties.k_set_starting_wages = k_set_starting_wages_value
        
        start_time = time.time()
        run_ant_algorithm(problem_properties, filename, method_of_probability_assignment=2)
        elapsed_time = time.time() - start_time
